In [17]:
# pip3 install pandas os numpy scikit-learn matplotlib seaborn torch 

In [74]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [19]:
metadata = pd.read_csv("data/gz2sample.csv.gz", compression="gzip")
metadata.to_csv("data/gz2sample.csv")
target_column_df = pd.read_csv("data/gz2_hart16.csv")

target column

In [45]:
target_col = target_column_df[["t04_spiral_a08_spiral_flag", "dr7objid"]]
target_col = target_col.rename(columns={'t04_spiral_a08_spiral_flag': 'target'})

In [46]:
target_col.head()

,target,dr7objid
0,1,587732591714893851
1,1,588009368545984617
2,0,587732484359913515
3,1,587741723357282317
4,0,587738410866966577


In [47]:
target_col.describe()

,target,dr7objid
count,239695.000000,2.396950e+05
mean,0.439588,5.878182e+17
std,0.496338,1.827461e+14
min,0.000000,5.877230e+17
25%,0.000000,5.877327e+17
50%,0.000000,5.877393e+17
75%,1.000000,5.877429e+17
max,1.000000,5.888489e+17


In [48]:
target_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239695 entries, 0 to 239694
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   target    239695 non-null  int64
 1   dr7objid  239695 non-null  int64
dtypes: int64(2)
memory usage: 3.7 MB


In [49]:
target_col.shape

(239695, 2)

In [50]:
null_target = target_col.isnull().sum()
print(null_target)

target      0
dr7objid    0
dtype: int64


In [51]:
flag_distribution = target_col.value_counts()
print(flag_distribution)

target  dr7objid          
0       587722981741363294    1
1       587732590105002040    1
        587732590104019094    1
        587732590104084621    1
        587732590104150155    1
                             ..
0       587739829274411045    1
        587739829274411084    1
        587739829274411113    1
        587739829274411129    1
1       588848901539627254    1
Name: count, Length: 239695, dtype: int64


metadata

In [52]:
metadata.head()

,OBJID,RUN,RERUN,CAMCOL,FIELD,OBJ,RA,DEC,PETROR50_R,PETROR90_R,...,PETROMAGERR_MG,PETROMAGERR_MR,PETROMAGERR_MI,PETROMAGERR_MZ,PETROR50_R_KPC,REGION,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN,WVT_BIN
0,587722981736054938,752,40,1,182,154,170.805579,-1.182146,4.486662,10.670979,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
1,587722981736120347,752,40,1,183,27,170.945892,-1.108325,13.407897,26.012415,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
2,587722981736579107,752,40,1,190,35,171.990936,-1.214009,8.175695,15.858483,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
3,587722981739069541,752,40,1,228,101,177.631858,-1.105126,2.929065,8.661545,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
4,587722981741363294,752,40,1,263,94,182.925262,-1.092357,2.695631,7.912694,...,0.016804,0.013804,0.014019,0.01994,3.973422,2,39,50,6,66


In [53]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325704 entries, 0 to 325703
Data columns (total 63 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   OBJID                      325704 non-null  int64  
 1   RUN                        325704 non-null  int64  
 2   RERUN                      325704 non-null  int64  
 3   CAMCOL                     325704 non-null  int64  
 4   FIELD                      325704 non-null  int64  
 5   OBJ                        325704 non-null  int64  
 6   RA                         325704 non-null  float64
 7   DEC                        325704 non-null  float64
 8   PETROR50_R                 325704 non-null  float64
 9   PETROR90_R                 325704 non-null  float64
 10  PETROMAG_U                 325704 non-null  float64
 11  PETROMAG_G                 325704 non-null  float64
 12  PETROMAG_R                 325704 non-null  float64
 13  PETROMAG_I                 32

In [54]:
metadata.describe()

,OBJID,RUN,RERUN,CAMCOL,FIELD,OBJ,RA,DEC,PETROR50_R,PETROR90_R,...,PETROMAGERR_MG,PETROMAGERR_MR,PETROMAGERR_MI,PETROMAGERR_MZ,PETROR50_R_KPC,REGION,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN,WVT_BIN
count,3.257040e+05,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,...,272542.000000,272542.000000,272542.000000,272542.000000,272542.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000
mean,1.340051e+18,3237.238486,36.718005,3.490009,217.833085,192.306803,184.034903,22.991662,3.249040,8.457078,...,3.702066,0.395241,6.631378,33.427876,4.311567,3.846348,-1594.583613,-1585.788299,-1576.455776,30.976681
std,2.344549e+18,1556.824795,11.155879,1.540730,164.813612,156.626991,72.178687,19.682874,24.874690,25.218483,...,605.719636,191.548567,812.646244,1826.954566,37.255247,2.679289,3712.106273,3716.071712,3728.635743,30.246290
min,5.877230e+17,106.000000,2.000000,1.000000,10.000000,1.000000,0.003800,-3.743851,-9999.000000,-9999.000000,...,0.010009,0.010008,0.010010,0.010050,-19408.213000,0.000000,-9999.000000,-9999.000000,-9999.000000,0.000000
25%,5.877328e+17,2243.000000,40.000000,2.000000,98.000000,105.000000,147.316952,3.798398,2.151452,5.917597,...,0.013134,0.012251,0.012697,0.019823,2.781042,2.000000,19.000000,28.000000,2.000000,2.000000
50%,5.877396e+17,3560.000000,40.000000,3.000000,171.000000,158.000000,186.195496,20.584429,2.819681,7.470993,...,0.015726,0.014031,0.014812,0.026565,4.006657,4.000000,35.000000,46.000000,5.000000,23.000000
75%,5.880155e+17,4512.000000,40.000000,5.000000,289.000000,231.000000,226.771229,38.181531,3.804628,9.785356,...,0.020539,0.017795,0.018936,0.037277,5.535648,6.000000,51.000000,62.000000,9.000000,52.000000
max,8.647475e+18,6793.000000,44.000000,6.000000,862.000000,2949.000000,359.995021,76.853188,106.211490,218.497340,...,99999.000000,99999.000000,99999.000000,99999.000000,104.335280,8.000000,999.000000,999.000000,999.000000,121.000000


In [55]:
metadata.shape

(325704, 63)

In [56]:
# creating new dataset

dataset_copy = metadata.copy()

dataset = pd.merge(metadata, target_col, left_on='OBJID', right_on='dr7objid', how="inner")

dataset.to_csv("data/metadata_updated.csv")

In [57]:
# checking which rows have null values, and how many

is_null_md = dataset.isnull().sum()

null_columns = is_null_md[is_null_md>0]
print(null_columns)

PETROMAG_MU    1
dtype: int64


In [58]:
# dropping rows with nulls values and checking how many were lost

dataset_before = dataset.copy()
dataset = dataset.dropna()
rows_dropped = dataset_before.shape[0]-dataset.shape[0]

print("Metadata dataset shape before: ", dataset_before.shape)
print("Metadata dataset shape after: ", dataset.shape)

print("Rows dropped: ", rows_dropped)

Metadata dataset shape before:  (239695, 65)
Metadata dataset shape after:  (239694, 65)
Rows dropped:  1


In [59]:
print(dataset.columns.tolist())

['OBJID', 'RUN', 'RERUN', 'CAMCOL', 'FIELD', 'OBJ', 'RA', 'DEC', 'PETROR50_R', 'PETROR90_R', 'PETROMAG_U', 'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I', 'PETROMAG_Z', 'PETROMAGERR_U', 'PETROMAGERR_G', 'PETROMAGERR_R', 'PETROMAGERR_I', 'PETROMAGERR_Z', 'PSFMAG_R', 'FIBERMAG_R', 'DEVMAG_R', 'DEVMAGERR_R', 'EXPMAG_R', 'EXPMAGERR_R', 'FRACDEV_R', 'MU50_R', 'EXTINCTION_U', 'EXTINCTION_G', 'EXTINCTION_R', 'EXTINCTION_I', 'EXTINCTION_Z', 'ROWC_U', 'COLC_U', 'ROWC_G', 'COLC_G', 'ROWC_R', 'COLC_R', 'ROWC_I', 'COLC_I', 'ROWC_Z', 'COLC_Z', 'CMODELMAG_R', 'CMODELMAGERR_R', 'REDSHIFT', 'REDSHIFTERR', 'PETROMAG_MU', 'PETROMAG_MG', 'PETROMAG_MR', 'PETROMAG_MI', 'PETROMAG_MZ', 'PETROMAGERR_MU', 'PETROMAGERR_MG', 'PETROMAGERR_MR', 'PETROMAGERR_MI', 'PETROMAGERR_MZ', 'PETROR50_R_KPC', 'REGION', 'PETROR50_R_KPC_SIMPLE_BIN', 'PETROMAG_MR_SIMPLE_BIN', 'REDSHIFT_SIMPLE_BIN', 'WVT_BIN', 'target', 'dr7objid']


In [60]:
# categorising data into numerical and categorical columns

df_dtypes = dataset.dtypes

numerical_cols = dataset.select_dtypes(include=np.number).columns.tolist()
categorical_cols = dataset.select_dtypes(exclude=np.number).columns.tolist()

numerical_df = df_dtypes[numerical_cols]
categorical_df = df_dtypes[categorical_cols]

print("Number of numerical columns: ", len(numerical_df))
print("Number of categorical columns: ", len(categorical_df))

Number of numerical columns:  65
Number of categorical columns:  0


In [61]:
# checking for duplicated rows

print(dataset.duplicated().sum())

0


In [62]:
# columns that show the error for corresponding columns, not needed for this model
error_cols = ['PETROMAGERR_U', 'PETROMAGERR_G', 'PETROMAGERR_R', 'PETROMAGERR_I', 'PETROMAGERR_Z', 'DEVMAGERR_R', 'EXPMAGERR_R', 'CMODELMAGERR_R', 'REDSHIFTERR', 'PETROMAGERR_MU', 'PETROMAGERR_MG', 'PETROMAGERR_MR', 'PETROMAGERR_MI', 'PETROMAGERR_MZ']

# columns that typically don't have any contribution to the identification of a galaxy
irrelevant_cols = ['ROWC_U', 'ROWC_G', 'ROWC_R', 'ROWC_I', 'ROWC_Z', 'COLC_U', 'COLC_G', 'COLC_R', 'COLC_I', 'COLC_Z', 'PETROR50_R_KPC_SIMPLE_BIN', 'PETROMAG_MR_SIMPLE_BIN', 'REDSHIFT_SIMPLE_BIN', 'WVT_BIN']

In [63]:
# dropping columns that are irrelevant and columns that show errors

cols_to_drop = error_cols + irrelevant_cols

data_before = dataset.copy()
dataset = dataset.drop(columns=cols_to_drop)
print(f"Columns dropped: {data_before.shape[1] - dataset.shape[1]}")
print("Features in reduced dataset: ", len(dataset.columns.tolist()))

Columns dropped: 28
Features in reduced dataset:  37


In [64]:
# number of columns

print(len(dataset.columns.tolist()))

37


In [65]:
# identifying columns with low variance

data_x = dataset.drop(columns=["target"])

selector = VarianceThreshold(threshold=0.1)
data_scaled = selector.fit_transform(data_x)

columns_to_keep = data_x.columns[selector.get_support()].tolist()
columns_to_drop = data_x.columns[~selector.get_support()].tolist()

print("Number of columns to drop: ", len(columns_to_drop))
print("Columns to drop: ", columns_to_drop)
print("Number of columns to keep: ", len(columns_to_keep))

Number of columns to drop:  6
Columns to drop:  ['EXTINCTION_U', 'EXTINCTION_G', 'EXTINCTION_R', 'EXTINCTION_I', 'EXTINCTION_Z', 'REDSHIFT']
Number of columns to keep:  30


Redshift is very important, however extinction can be dropped

In [66]:
# removing columns with low variance 

columns_to_drop.remove('REDSHIFT')

dataset = dataset.drop(columns=columns_to_drop)

In [67]:
print(dataset.shape)

(239694, 32)


# Models

In [68]:
x = dataset.drop(columns=["target"])
y = dataset["target"]

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=14)

In [71]:
# Random Forest Classifier

rf = RandomForestClassifier(random_state=20)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)

print(f"Accuracy: {accuracy_score(y_pred_rf, y_test)*100:.2f}%")
print("Recall Score: ", recall_score(y_pred_rf, y_test))
print("Precision Score: ", precision_score(y_pred_rf, y_test))
print("F1 Score: ", f1_score(y_pred_rf, y_test))
print("Confusion Matrix", confusion_matrix(y_pred_rf, y_test))

Accuracy: 78.22%
Recall Score:  0.8039798719121684
Precision Score:  0.6672361427486713
F1 Score:  0.729253112033195
Confusion Matrix [[23439  7012]
 [ 3428 14060]]


In [73]:
# Gradient Boosting Classifier 

gb = GradientBoostingClassifier(random_state=92)
gb.fit(x_train, y_train)
y_pred_gb = gb.predict(x_test)

print(f"Accuracy: {accuracy_score(y_pred_gb, y_test)*100:.2f}%")
print("Recall Score: ", recall_score(y_pred_gb, y_test))
print("Precision Score: ", precision_score(y_pred_gb, y_test))
print("F1 Score: ", f1_score(y_pred_gb, y_test))
print("Confusion Matrix", confusion_matrix(y_pred_gb, y_test))

Accuracy: 77.61%
Recall Score:  0.7976965159804204
Precision Score:  0.6573652239939256
F1 Score:  0.7207638473345995
Confusion Matrix [[23354  7220]
 [ 3513 13852]]


In [75]:
# MLP Classifier 

mlp = MLPClassifier(random_state=75)
mlp.fit(x_train, y_train)
y_pred_mlp = mlp.predict(x_test)

print(f"Accuracy: {accuracy_score(y_pred_mlp, y_test)*100:.2f}%")
print("Recall Score: ", recall_score(y_pred_mlp, y_test))
print("Precision Score: ", precision_score(y_pred_mlp, y_test))
print("F1 Score: ", f1_score(y_pred_mlp, y_test))
print("Confusion Matrix", confusion_matrix(y_pred_mlp, y_test))

Accuracy: 79.10%
Recall Score:  0.8261998937363481
Precision Score:  0.6641514806378133
F1 Score:  0.7363657888505959
Confusion Matrix [[23923  7077]
 [ 2944 13995]]


In [76]:
# Logistic Regression 

log = LogisticRegression(random_state=4, max_iter=1000)
log.fit(x_train, y_train)
y_pred_log = log.predict(x_test)

print(f"Accuracy: {accuracy_score(y_pred_log, y_test)*100:.2f}%")
print("Recall Score: ", recall_score(y_pred_log, y_test))
print("Precision Score: ", precision_score(y_pred_log, y_test))
print("F1 Score: ", f1_score(y_pred_log, y_test))
print("Confusion Matrix", confusion_matrix(y_pred_log, y_test))

Accuracy: 76.96%
Recall Score:  0.7733246087572933
Precision Score:  0.6730258162490509
F1 Score:  0.7196975463703027
Confusion Matrix [[22710  6890]
 [ 4157 14182]]
